In [16]:
%%capture
!pip install datasets transformers[sentencepiece]
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install scipy sklearn

In [3]:
import torch

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
device

device(type='cuda')

In [6]:
!nvidia-smi

Tue May 17 09:04:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.01       Driver Version: 516.01       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   51C    P8     5W /  N/A |    419MiB /  6144MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import transformers
print(transformers.__version__)

4.19.1


# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the 🤗 Transformers model to a text classification task of the GLUE Benchmark.

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

CoLA (Corpus of Linguistic Acceptability) : Determine if a sentence is grammatically correct or not.is a dataset containing sentences labeled grammatically correct or not.

MNLI (Multi-Genre Natural Language Inference) : Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)

MRPC (Microsoft Research Paraphrase Corpus) : Determine if two sentences are paraphrases from one another or not.

QNLI (Question-answering Natural Language Inference) : Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)

QQP (Quora Question Pairs2) : Determine if two questions are semantically equivalent or not.

RTE (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.

SST-2 (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.

STS-B (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.

WNLI (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the Trainer API to fine-tune a model on it. Each task is named by its acronym, with mnli-mm standing for the mismatched version of MNLI (so same training set as mnli but different validation and test sets):

In [10]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the Model Hub as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [15]:
task = "mnli"
model_checkpoint = "distilbert-base-uncased" ##choose distilbert to reduce training time. bert,roberta,deberta all the models based on transformer encoder architecture with classification head on top can be used
batch_size = 16

# Loading the dataset

We will use the 🤗 Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions load_dataset and load_metric.

In [13]:
from datasets import load_dataset,load_metric

Apart from mnli-mm being a special code, we can directly pass our task name to those functions. load_dataset will cache the dataset to avoid downloading it again the next time you run this cell.

In [17]:
actual_task = "mnli" if task == "mnli-mm" else task
print(f"going to load {actual_task} dataset as well as metric for evaluation")
dataset = load_dataset('glue',actual_task)
metric = load_metric('glue',actual_task)

going to load mnli dataset as well as metric for evaluation


Reusing dataset glue (C:\Users\ghosh\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

In [40]:
print(f"loaded dataset : {dataset}")

loaded dataset : DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


let's explore the dataset

In [41]:
dataset['train'].features #features of the dataset

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [49]:
#let's print integer label corresponda to string labels
for _ in ['entailment', 'neutral', 'contradiction']:
    print(f"integer label for  {_} :: {dataset['train'].features['label'].str2int(_)}")

integer label for  entailment :: 0
integer label for  neutral :: 1
integer label for  contradiction :: 2


In [50]:
dataset['train'][0] # first element of the dataset

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

In [47]:
dataset['train'][:5]

{'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.',
  'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him',
  'One of our number will carry out your instructions minutely.',
  'How do you know? All this is their information again.',
  "yeah i tell you what though if you go price some of those tennis shoes i can see why now you know they're getting up in the hundred dollar range"],
 'hypothesis': ['Product and geography are what make cream skimming work. ',
  'You lose the things to the following level if the people recall.',
  'A member of my team will execute your orders with immense precision.',
  'This information belongs to them.',
  'The tennis shoes have a range of prices.'],
 'label': [1, 0, 0, 0, 1],
 'idx': [0,

In [56]:
print(dataset['train'][:5].keys())
for k,v in dataset['train'][:5].items():
    print(f"shape of {k} : {len(v)}")

dict_keys(['premise', 'hypothesis', 'label', 'idx'])
shape of premise : 5
shape of hypothesis : 5
shape of label : 5
shape of idx : 5


In [57]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

You can call its compute method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [58]:
metric.compute?? ## this is how you need to see the documentation in jupyter

In [63]:
import numpy as np
fake_preds = np.random.randint(0, 3, size=(64,)) # as in MNLI dataset we have 3 labels, so here I am generating 64 numbers between 0 and 3
fake_labels = np.random.randint(0, 3, size=(64,))
metric.compute(predictions=fake_preds,)